In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import MissingValuesWarning
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [8]:
y_datetime = pd.Series(
    np.arange(15), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='y', dtype=float
)

rolling = RollingFeatures(
    stats=['mean', 'median', 'sum'], window_sizes=[5, 5, 6]
)

results = rolling.transform_batch(y_datetime)
results

,roll_mean_5,roll_median_5,roll_sum_6
2000-01-01,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN
2000-01-05,2.0,2.0,NaN
2000-01-06,3.0,3.0,15.0
2000-01-07,4.0,4.0,21.0
2000-01-08,5.0,5.0,27.0
2000-01-09,6.0,6.0,33.0
2000-01-10,7.0,7.0,39.0


In [9]:
results.iloc[6:]

,roll_mean_5,roll_median_5,roll_sum_6
2000-01-07,4.0,4.0,21.0
2000-01-08,5.0,5.0,27.0
2000-01-09,6.0,6.0,33.0
2000-01-10,7.0,7.0,39.0
2000-01-11,8.0,8.0,45.0
2000-01-12,9.0,9.0,51.0
2000-01-13,10.0,10.0,57.0
2000-01-14,11.0,11.0,63.0
2000-01-15,12.0,12.0,69.0


In [14]:
y_datetime = pd.Series(
    np.arange(15), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='y', dtype=float
)
exog_datetime = pd.Series(
    np.arange(100, 115), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='exog', dtype=float
)
forecaster = ForecasterAutoreg(LinearRegression(), lags=5, window_features=rolling)
results = forecaster._create_train_X_y(y=y_datetime, exog=exog_datetime)

In [18]:
y_datetime

2000-01-01     0.0
2000-01-02     1.0
2000-01-03     2.0
2000-01-04     3.0
2000-01-05     4.0
2000-01-06     5.0
2000-01-07     6.0
2000-01-08     7.0
2000-01-09     8.0
2000-01-10     9.0
2000-01-11    10.0
2000-01-12    11.0
2000-01-13    12.0
2000-01-14    13.0
2000-01-15    14.0
Freq: D, Name: y, dtype: float64

In [17]:
forecaster._create_lags(y=y_datetime.to_numpy(), X_as_pandas=True, 
                        train_index=y_datetime.index[forecaster.window_size:])

(            lag_1  lag_2  lag_3  lag_4  lag_5
 2000-01-07    5.0    4.0    3.0    2.0    1.0
 2000-01-08    6.0    5.0    4.0    3.0    2.0
 2000-01-09    7.0    6.0    5.0    4.0    3.0
 2000-01-10    8.0    7.0    6.0    5.0    4.0
 2000-01-11    9.0    8.0    7.0    6.0    5.0
 2000-01-12   10.0    9.0    8.0    7.0    6.0
 2000-01-13   11.0   10.0    9.0    8.0    7.0
 2000-01-14   12.0   11.0   10.0    9.0    8.0
 2000-01-15   13.0   12.0   11.0   10.0    9.0,
 array([ 6.,  7.,  8.,  9., 10., 11., 12., 13., 14.]))

In [6]:

y_datetime = pd.Series(
    np.arange(10),pd.date_range(start='2020-01-01', periods=10)
)
train_index = pd.date_range(start='2020-01-06', periods=5)
rolling = RollingFeatures(
    stats=['mean', 'median', 'sum'], window_sizes=[5, 5, 6]
)

forecaster = ForecasterAutoreg(LinearRegression(), lags=3,
                                window_features=rolling)
results = forecaster._create_window_features(
    y=y_datetime, train_index=train_index, X_as_pandas=True
)
results[0]

            roll_mean_5  roll_median_5  roll_sum_6
2020-01-06          3.0            3.0        15.0
2020-01-07          4.0            4.0        21.0
2020-01-08          5.0            5.0        27.0
2020-01-09          6.0            6.0        33.0
2020-01-10          7.0            7.0        39.0
DatetimeIndex(['2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09',
               '2020-01-10'],
              dtype='datetime64[ns]', freq='D')


[            roll_mean_5  roll_median_5  roll_sum_6
 2020-01-06          3.0            3.0        15.0
 2020-01-07          4.0            4.0        21.0
 2020-01-08          5.0            5.0        27.0
 2020-01-09          6.0            6.0        33.0
 2020-01-10          7.0            7.0        39.0]